# best

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 16

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [6]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 0):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=3,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.25,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [7]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [8]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64

In [9]:

def feature_engineer(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)

       # 결측치
       df = df.replace(' ', np.nan)
       df = df.fillna(0)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       market_cols = ['단가(원)', '경매건수', '거래량', '거래대금(원)'] # 

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = ['하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '해당일자_전체거래물량(kg)',
       '일자별_도매가격_최대(원)', '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)', 
       '하위가격 평균가(원)', '상위가격 평균가(원)' ] 
       # 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['datadate', 'cos_time'] # ,, '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = [] 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열

       df = df[[c for c in df.columns if c in selected_columns]]

       return df

&nbsp;

## Train 과정

In [10]:
from sklearn.preprocessing import MinMaxScaler

losses = []
scalers = {}

for i in tqdm(data_list):
    print(i)
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    # 사용할 열 선택 및 index 설정
    df = feature_engineer(df)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.replace({' ': np.nan})
    df = df.fillna(0)
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.001, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]

./aT_data/data/train/train_4.csv
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-21 18:23:42.111263: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-21 18:23:42.111832: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-21 18:23:42.920881: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-21 18:23:43.097232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 18:23:43.587583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  3%|▎         | 1/37 [01:37<58:14, 97.06s/it]


valid_loss:  751001.4375
./aT_data/data/train/train_15.csv


2022-09-21 18:25:18.917291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 18:25:19.128467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  5%|▌         | 2/37 [03:19<58:21, 100.06s/it]

...


valid_loss:  72550.8984375
./aT_data/data/train/train_18.csv


2022-09-21 19:23:42.050103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:23:42.317915: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 97%|█████████▋| 36/37 [1:01:33<01:32, 92.66s/it]


valid_loss:  106420168.0
./aT_data/data/train/train_9.csv


2022-09-21 19:25:15.633249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:25:16.389723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 37/37 [1:02:59<00:00, 102.16s/it]


valid_loss:  143529.59375


In [11]:
losses

[751001.4375,
 396421.90625,
 208970.03125,
 47639.41015625,
 23399532.0,
 4107801.25,
 5155220.0,
 2243115.75,
 1547819.375,
 70643728.0,
 845366.75,
 694899.5625,
 568734.1875,
 1433096.0,
 155009.0,
 184306.21875,
 5128294.5,
 2973533.75,
 3618560.5,
 585460.375,
 247897.859375,
 828530.125,
 6296029.0,
 345366.03125,
 129421.40625,
 2562734.25,
 2345519.75,
 6458560.0,
 9804542.0,
 2352154.0,
 567381.9375,
 3793452.0,
 3878465.75,
 269063.5625,
 72550.8984375,
 106420168.0,
 143529.59375]

&nbsp;

## Test 과정

In [12]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택 및 index 설정

        df = feature_engineer(df)
        df = df[[c for c in df.columns if c != '해당일자_전체평균가격(원)']]

        df.set_index('datadate', drop=True, inplace=True)
        
        # # nan 처리
        # df = df.replace({' ': np.nan})
        # df = df.fillna(0)

        # # train input 과 형상 맞추기
        # add_col = [i for i in check_col if i not in df.columns]

        # for a in add_col:
        #     df[a] = 0

        # # ' ' -> nan 으로 변경
        # for a in df.columns:
        #     df[a] = df[a].replace({' ': np.nan})

        # # nan 처리
        # df = df.fillna(0)
        # # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 173ms/step


2022-09-21 19:26:41.736471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:26:41.934743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 28ms/step


2022-09-21 19:26:42.015115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:26:42.096197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:26:42.175212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-21 19:27:15.180607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-21 19:27:15.272113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 32ms/step


2022-09-21 19:27:15.363003: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 37ms/step


2022-09-21 19:27:15.455929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [00:33<00:00,  3.40s/it]


&nbsp;

## 정답 제출 파일생성

In [13]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  9.15it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [14]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-653.192930,-337.808780,-385.908480,-599.413400,-527.448800,-2168.829600,-52.297066,-421.581360,-675.544070,-77.273240,...,-2580.742200,-48.784070,-159.180500,-202.573230,-955.224300,-1104.650400,-1593.848000,-1634.649500,-504.297240,-372.797800
2,-1419.783900,-703.094500,-1317.136700,-1438.783700,-1557.982000,-2995.400600,-103.376310,-969.470460,-2079.054000,-508.053620,...,-3668.422400,-263.462500,-459.841000,-452.780150,-1413.737400,-1658.824600,-2273.903300,-2074.212400,-945.634900,-1034.361100
3,-1778.965600,-631.046750,-1631.389200,-1488.176800,-1801.011600,-2621.625200,-106.130850,-1243.711700,-2589.169200,-967.848630,...,-3439.190200,-360.707520,-595.614000,-857.200260,-1275.463100,-1690.701400,-2183.155000,-1919.146200,-1086.967000,-1331.966600
4,-1074.995000,-369.045960,-617.541200,-867.477500,-999.585200,-1945.049700,-106.071560,-738.617250,-1209.561800,-261.019350,...,-3270.968300,-135.758600,-193.936740,-254.021260,-1187.499500,-1164.077800,-1542.617700,-1671.553100,-581.949900,-415.906650
5,-82.964294,-239.864690,201.330300,-610.336240,-357.367700,-2003.842200,-86.626526,-192.218700,-167.764470,56.395443,...,-3380.252400,3.462398,22.839386,-46.669150,-1206.140400,-804.513550,-1297.763500,-1509.499000,-216.501040,-3.602535
6,-101.339630,-193.165940,-70.949520,-473.389040,-294.450600,-1258.063100,-70.046470,-82.477610,-261.234680,-116.260970,...,-2440.056600,-42.156640,-57.334880,-173.462200,-836.271100,-523.126800,-611.777300,-1052.912800,-229.099040,-129.725080
7,-122.393900,-49.349100,-75.314330,-258.525760,-251.140660,-249.136400,-36.265198,-96.613050,-218.920800,-42.428486,...,-499.511380,-32.661556,-70.616630,-111.544890,-49.221460,-170.982700,-157.814380,-283.388180,-133.209930,-89.134254
8,-766.792300,-374.199980,-462.983500,-896.262150,-762.578000,-2222.851800,-57.751520,-437.620400,-786.860050,-107.122820,...,-2724.598900,-74.452934,-208.250780,-259.613830,-929.497400,-1226.998900,-1624.148400,-1676.273600,-643.568500,-421.715600
9,-1431.010600,-660.670840,-1321.953700,-1608.741200,-1664.486500,-2927.709000,-65.952340,-958.062000,-2079.980500,-542.912900,...,-3631.552500,-270.338840,-489.948600,-514.383670,-1309.827000,-1726.091900,-2282.455000,-2034.299600,-962.543900,-1046.371000


- 변동률 계산 

In [15]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값
    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)
    
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [16]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [17]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.168735,-0.248003,-0.132624,-0.176294,-0.133605,-0.234368,-0.019246,-0.125432,-0.137532,-0.065876,...,-0.144001,-0.129334,-0.262896,-0.202513,-0.283644,-0.312515,-0.173590,-0.178571,0,d+1
1,-0.366763,-0.516178,-0.452658,-0.423162,-0.394645,-0.323689,-0.038044,-0.288444,-0.423269,-0.433116,...,-0.415990,-0.289081,-0.389088,-0.304109,-0.404668,-0.396552,-0.325508,-0.495461,0,d+2
2,-0.459547,-0.463284,-0.560657,-0.437689,-0.456205,-0.283298,-0.039058,-0.370039,-0.527122,-0.825092,...,-0.538816,-0.547286,-0.351032,-0.309953,-0.388518,-0.366906,-0.374158,-0.638014,0,d+3
3,-0.277696,-0.270935,-0.212229,-0.255135,-0.253200,-0.210186,-0.039036,-0.219759,-0.246251,-0.222519,...,-0.175443,-0.162182,-0.326823,-0.213408,-0.274527,-0.319571,-0.200320,-0.199220,0,d+4
4,-0.021432,-0.176097,0.069191,-0.179507,-0.090523,-0.216539,-0.031880,-0.057190,-0.034155,0.048077,...,0.020661,-0.029796,-0.331953,-0.147490,-0.230952,-0.288589,-0.074524,-0.001726,0,d+5
5,-0.026178,-0.141813,-0.024383,-0.139229,-0.074586,-0.135949,-0.025778,-0.024539,-0.053184,-0.099113,...,-0.051867,-0.110748,-0.230158,-0.095904,-0.108873,-0.201298,-0.078861,-0.062139,0,d+6
6,-0.031617,-0.036230,-0.025883,-0.076035,-0.063615,-0.026922,-0.013346,-0.028745,-0.044569,-0.036170,...,-0.063883,-0.071217,-0.013547,-0.031346,-0.028085,-0.054179,-0.045854,-0.042695,0,d+7
7,-0.198080,-0.274719,-0.159113,-0.263601,-0.193165,-0.240206,-0.021253,-0.130204,-0.160195,-0.091322,...,-0.188392,-0.165752,-0.255816,-0.224943,-0.289036,-0.320473,-0.221531,-0.202003,0,d+8
8,-0.369663,-0.485032,-0.454313,-0.473149,-0.421623,-0.316374,-0.024271,-0.285050,-0.423457,-0.462834,...,-0.443227,-0.328412,-0.360490,-0.316441,-0.406189,-0.388921,-0.331329,-0.501214,0,d+9
9,-0.435062,-0.438777,-0.555990,-0.470348,-0.470982,-0.272474,-0.027575,-0.364257,-0.522950,-0.824305,...,-0.560661,-0.559910,-0.325682,-0.316833,-0.380283,-0.360925,-0.348827,-0.619394,0,d+10


- 정답 양식으로 변경

In [18]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [19]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.168735,-0.248003,-0.132624,-0.176294,-0.133605,-0.234368,-0.019246,-0.125432,...,-0.298669,-0.081036,-0.144001,-0.129334,-0.262896,-0.202513,-0.283644,-0.312515,-0.173590,-0.178571
1,0,d+2,-0.366763,-0.516178,-0.452658,-0.423162,-0.394645,-0.323689,-0.038044,-0.288444,...,-0.424546,-0.437641,-0.415990,-0.289081,-0.389088,-0.304109,-0.404668,-0.396552,-0.325508,-0.495461
2,0,d+3,-0.459547,-0.463284,-0.560657,-0.437689,-0.456205,-0.283298,-0.039058,-0.370039,...,-0.398017,-0.599176,-0.538816,-0.547286,-0.351032,-0.309953,-0.388518,-0.366906,-0.374158,-0.638014
3,0,d+4,-0.277696,-0.270935,-0.212229,-0.255135,-0.253200,-0.210186,-0.039036,-0.219759,...,-0.378549,-0.225511,-0.175443,-0.162182,-0.326823,-0.213408,-0.274527,-0.319571,-0.200320,-0.199220
4,0,d+5,-0.021432,-0.176097,0.069191,-0.179507,-0.090523,-0.216539,-0.031880,-0.057190,...,-0.391196,0.005751,0.020661,-0.029796,-0.331953,-0.147490,-0.230952,-0.288589,-0.074524,-0.001726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.099853,-0.135005,-0.122358,-0.110801,-0.232440,-0.000693,-0.025300,NaN,...,-0.070942,0.026734,-0.025695,-0.068883,-0.132496,-0.123975,-0.149744,-0.038738,-0.093153,-0.099125
146,9,d+12,0.101821,-0.040765,0.179026,0.020201,-0.010853,0.014032,-0.024648,NaN,...,-0.070209,0.218171,0.184992,0.108597,-0.116553,0.053875,-0.100259,0.009645,0.106734,0.135604
147,9,d+13,0.049650,-0.120818,-0.098204,-0.021705,-0.022705,0.005970,-0.021936,NaN,...,-0.063361,-0.076957,-0.050404,-0.066743,-0.094723,0.045420,-0.065178,0.015221,0.018391,-0.113778
148,9,d+14,0.017433,-0.013930,0.028916,-0.009063,-0.015571,0.003025,-0.011830,NaN,...,-0.027814,-0.013639,0.046772,0.013275,0.010806,0.000906,0.009602,0.011987,-0.011837,-0.004762


- 제출 양식

In [20]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [21]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.168735,-0.248003,-0.132624,-0.176294,-0.133605,-0.234368,-0.019246,-0.125432,...,-0.298669,-0.081036,-0.144001,-0.129334,-0.262896,-0.202513,-0.283644,-0.312515,-0.173590,-0.178571
1,0,d+2,-0.366763,-0.516178,-0.452658,-0.423162,-0.394645,-0.323689,NaN,-0.288444,...,-0.424546,-0.437641,-0.415990,-0.289081,-0.389088,-0.304109,-0.404668,-0.396552,-0.325508,-0.495461
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.039058,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.277696,-0.270935,-0.212229,-0.255135,-0.253200,-0.210186,NaN,-0.219759,...,-0.378549,-0.225511,-0.175443,-0.162182,-0.326823,-0.213408,-0.274527,-0.319571,-0.200320,-0.199220
4,0,d+5,-0.021432,-0.176097,0.069191,-0.179507,-0.090523,-0.216539,NaN,-0.057190,...,-0.391196,0.005751,0.020661,-0.029796,-0.331953,-0.147490,-0.230952,-0.288589,-0.074524,-0.001726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.099853,-0.135005,-0.122358,-0.110801,-0.232440,-0.000693,-0.025300,NaN,...,-0.070942,0.026734,-0.025695,-0.068883,-0.132496,-0.123975,-0.149744,-0.038738,-0.093153,-0.099125
146,9,d+12,0.101821,-0.040765,0.179026,0.020201,-0.010853,0.014032,-0.024648,NaN,...,-0.070209,0.218171,0.184992,0.108597,-0.116553,0.053875,-0.100259,0.009645,0.106734,0.135604
147,9,d+13,0.049650,-0.120818,-0.098204,-0.021705,-0.022705,0.005970,-0.021936,NaN,...,-0.063361,-0.076957,-0.050404,-0.066743,-0.094723,0.045420,-0.065178,0.015221,0.018391,-0.113778
148,9,d+14,0.017433,-0.013930,0.028916,-0.009063,-0.015571,0.003025,-0.011830,NaN,...,-0.027814,-0.013639,0.046772,0.013275,0.010806,0.000906,0.009602,0.011987,-0.011837,-0.004762
